In [1]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from collections import namedtuple
from pyspark import SparkConf,SparkContext
from pyspark.sql import Row,SQLContext
import sys
import requests

# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")

sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

sqlContext = SQLContext(sc)
ssc = StreamingContext(sc,10)
ssc.checkpoint("checkpoint_TwitterApp")
socket_stream = ssc.socketTextStream("localhost", 5555)


In [ ]:
lines = socket_stream.window(20)
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

(lines.flatMap( lambda text: text.split( " " ) )
.filter( lambda word: word.lower().startswith("#") )
.map( lambda word: ( word.lower(), 1 ) )
.reduceByKey( lambda a, b: a + b )
.map( lambda rec: Tweet( rec[0], rec[1] ) )
.foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") )
.limit(10).registerTempTable("tweets") ) )

ssc.start()

In [ ]:
dataStream = ssc.socketTextStream("localhost",5555)
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

----------- 2020-08-01 15:10:50 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:11:00 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:11:10 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:11:20 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:11:30 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:11:40 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:11:50 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:12:00 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:12:10 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:12:20 -----------
Error: <class 'ValueError'>
----------- 2020-08-01 15:12:30 -----------
+-----------------+-------------+
|          hashtag|hashtag_count|
+-----------------+-------------+
|          #Africa|            1|
|           #Japan|            1|
|           #India|            1|
|      #Indiafirst|            1

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|             #webdev|            4|
|         #javascript|            4|
|             #coding|            2|
|#ArtificialIntell...|            2|
|               #tech|            2|
|           #technews|            2|
|         #technology|            2|
|        #CodeNewbies|            2|
|        #programming|            2|
|               #Data|            2|
+--------------------+-------------+

Error: <class 'NameError'>
----------- 2020-08-01 15:15:00 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|             #webdev|            4|
|         #javascript|            4|
|             #coding|            2|
|#ArtificialIntell...|            2|
|               #tech|            2|
|           #technews|            2|
|         #technology|            2|
|        #CodeNewbies|            2|
|  

+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|             #webdev|            4|
|         #javascript|            4|
|               #java|            3|
|           #technews|            2|
|         #technology|            2|
|               #tech|            2|
|        #CodeNewbies|            2|
|             #coding|            2|
|        #programming|            2|
|#ArtificialIntell...|            2|
+--------------------+-------------+

Error: <class 'NameError'>
----------- 2020-08-01 15:17:20 -----------
+--------------------+-------------+
|             hashtag|hashtag_count|
+--------------------+-------------+
|             #webdev|            4|
|         #javascript|            4|
|               #java|            3|
|           #technews|            2|
|         #technology|            2|
|               #tech|            2|
|        #CodeNewbies|            2|
|             #coding|            2|
|  

+--------------+-------------+
|       hashtag|hashtag_count|
+--------------+-------------+
|         #java|           21|
|   #javascript|           20|
|       #Coding|           12|
|       #Python|           12|
|      #ReactJS|           11|
|          #AWS|           11|
|       #rstats|           11|
|#100DaysOfCode|           11|
|       #Clo…RT|           10|
|       #coding|            7|
+--------------+-------------+

Error: <class 'NameError'>
----------- 2020-08-01 15:20:00 -----------
+--------------+-------------+
|       hashtag|hashtag_count|
+--------------+-------------+
|         #java|           21|
|   #javascript|           20|
|       #Coding|           12|
|       #Python|           12|
|      #ReactJS|           11|
|          #AWS|           11|
|       #rstats|           11|
|#100DaysOfCode|           11|
|       #Clo…RT|           10|
|       #coding|            7|
+--------------+-------------+

Error: <class 'NameError'>
----------- 2020-08-01 15:20:10 

+--------------+-------------+
|       hashtag|hashtag_count|
+--------------+-------------+
|         #java|           26|
|   #javascript|           25|
|       #Coding|           17|
|       #Python|           16|
|          #AWS|           15|
|      #ReactJS|           15|
|       #rstats|           15|
|#100DaysOfCode|           15|
|       #Clo…RT|           13|
|         #Java|            8|
+--------------+-------------+

Error: <class 'NameError'>
----------- 2020-08-01 15:22:50 -----------
+--------------+-------------+
|       hashtag|hashtag_count|
+--------------+-------------+
|         #java|           26|
|   #javascript|           25|
|       #Coding|           17|
|       #Python|           16|
|          #AWS|           15|
|      #ReactJS|           15|
|       #rstats|           15|
|#100DaysOfCode|           15|
|       #Clo…RT|           13|
|         #Java|            8|
+--------------+-------------+

Error: <class 'NameError'>
----------- 2020-08-01 15:23:00 

In [3]:
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

In [4]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']


In [5]:

def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [ ]:
def send_df_to_console(df):
    # extract the hashtags from dataframe and convert them into array
    top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
    # extract the counts from dataframe and convert them into array
    tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
    # initialize and send the data through REST API
    url = 'http://localhost:5000/updateData'
    request_data = {'label': str(top_tags), 'data': str(tags_count)}
    response = requests.post(url, data=request_data)
    print("Details",response,request_data)

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline # Only works for Jupyter Notebooks!
count = 0
while count < 10:
    
    time.sleep( 3 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    print(top_10_tweets)
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure( figsize = ( 10, 8 ) )
    sns.barplot( x="count", y="tag", data=top_10_df)
    sns.plt.show()
    count = count + 1